# Pulling data from the Opta Vision Remote (MA35) Feed

This notebook outlines a basic framework to pull event-by-event data enriched with Opta Vision from the MA35 SDAPI feed.

The feed particularily contains information about passes as created by our pass models xPass, xReceiver, and xThreat:
- **xPass** is the likelihood that a pass will be completed to a player
- **xReceiver** is the likelihood that the ball carrier is passing to a player
- **xThreat** is the likelihood that a pass to a player will be followed by a shot within the next 10 seconds

To be able to run this notebook properly please make sure that the file 'qualifier_names.csv' is located in the same folder as the notebook.

In [2]:
import pandas as pd
import json
import os
import numpy as np

pd.options.display.max_columns = 999

## Load Opta Vision data

In [4]:
# get the path of the json file
current_directory = os.getcwd()
base_path = os.path.join(os.path.join(os.path.dirname(current_directory),"data"),'first_10_events')

game_id = 1
file_name = f'{game_id}.json'

# load data from file
with open(f'{base_path}/{file_name}') as f:
    data=json.loads(f.read())
    
f.close()
 
# transform data into pandas dataframe
events = pd.json_normalize(data['liveData']['event'])

# make a copy of it for later usage
events_all = events.copy()

print(events.isna().sum())
print(events.shape)

id                                          0
eventId                                     0
typeId                                      0
periodId                                    0
timeMin                                     0
timeSec                                     0
contestantId                                0
outcome                                     0
x                                           0
y                                           0
timeStamp                                   0
lastModified                                0
qualifier                                   0
playerId                                   29
lineBreakingPass.linesBroken.value       1394
passOption.player                         860
passTarget.player                         860
xThreat.applied                          1028
lineBreakingPass.lastLineBroken.value    1693
pressure.pressureReceived.value          1002
pressure.player                          1002
xThreat.removed                   

## Get Player Info

In [21]:
# merge on the player names from the Opta Vision data
#player_info = events_all[['playerId', 'playerName']].drop_duplicates()

## Get qualifier info

To do deeper analysis we're going to need qualifier info on each event.

Qualifier info is actually stored in the feed as a nested dictionary. We will merge on the qualifier names from a local dataset for ease of use here.

This method below shows how to use this qualifier info. However, there remain some drawbacks:
- This is also an expensive operation. For a full PL season I expect ~4 million qualifiers - which could be rather slow to handle.

In [6]:
# read in qualifier list
path_data = os.path.join(os.path.dirname(current_directory),'data')
qualifier_names = pd.read_csv(os.path.join(path_data,"qualifier_names.csv"))

In [7]:
# explode coverts each element in each list to a separate row
cols = ['id', 'qualifier']
qualifiers = events_all[cols].explode('qualifier')
display(qualifiers)

,id,qualifier
0,2423529553,"{'id': 3585971337, 'qualifierId': 59, 'value':..."
0,2423529553,"{'id': 3585971333, 'qualifierId': 30, 'value':..."
0,2423529553,"{'id': 3585971341, 'qualifierId': 131, 'value'..."
0,2423529553,"{'id': 3585971335, 'qualifierId': 44, 'value':..."
0,2423529553,"{'id': 3585971345, 'qualifierId': 227, 'value'..."
...,...,...
1790,2423666413,"{'id': 3586731763, 'qualifierId': 209}"
1790,2423666413,"{'id': 3586731761, 'qualifierId': 57, 'value':..."
1791,2423666421,"{'id': 3587799179, 'qualifierId': 406, 'value'..."
1791,2423666421,"{'id': 3586752763, 'qualifierId': 302}"


In [31]:
# remove NA
qualifiers = qualifiers[qualifiers.qualifier.notna()].reset_index(drop=True)
print(qualifiers.shape)

(9456, 2)


In [32]:
# save corresponding event ids for each qualifier
event_ids = qualifiers.id.tolist()

In [33]:
qualifiers = pd.json_normalize(qualifiers[qualifiers.qualifier.notna()]['qualifier'])
print(qualifiers.shape)
display(qualifiers.head())

(9456, 3)


,id,qualifierId,value
0,3585971337,59,"12, 23, 13, 4, 5, 16, 11, 22, 17, 1, 7, 19, 14..."
1,3585971333,30,"a56woizbe4g6jpl3fg4tlgno5, 4u281v53ges3kimtgac..."
2,3585971341,131,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0,..."
3,3585971335,44,"1, 2, 2, 3, 2, 2, 3, 3, 4, 4, 3, 5, 5, 5, 5, 5..."
4,3585971345,227,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [36]:
qualifiers['event_id'] = event_ids
display(qualifiers.head())

,id,qualifierId,value,event_id
0,3585971337,59,"12, 23, 13, 4, 5, 16, 11, 22, 17, 1, 7, 19, 14...",2423529553
1,3585971333,30,"a56woizbe4g6jpl3fg4tlgno5, 4u281v53ges3kimtgac...",2423529553
2,3585971341,131,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0,...",2423529553
3,3585971335,44,"1, 2, 2, 3, 2, 2, 3, 3, 4, 4, 3, 5, 5, 5, 5, 5...",2423529553
4,3585971345,227,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2423529553


In [37]:
qualifiers = qualifiers.merge(qualifier_names, how='left', on='qualifierId')

In [38]:
qualifiers.head()

,id,qualifierId,value,event_id,qualifier
0,3585971337,59,"12, 23, 13, 4, 5, 16, 11, 22, 17, 1, 7, 19, 14...",2423529553,Jersey Number
1,3585971333,30,"a56woizbe4g6jpl3fg4tlgno5, 4u281v53ges3kimtgac...",2423529553,Involved
2,3585971341,131,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0,...",2423529553,Team Player Formation
3,3585971335,44,"1, 2, 2, 3, 2, 2, 3, 3, 4, 4, 3, 5, 5, 5, 5, 5...",2423529553,Player position
4,3585971345,227,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",2423529553,Resume


### Keep only Pass End qualifiers (for now)

To expand this, add the qualifier ids we need.

In [39]:
# in the qualifier_names.csv, the id of pass_end_x and pass_end_y are 140 and 141
pass_end_qualifiers = qualifiers[qualifiers.qualifierId.isin([140,141])].copy()

In [40]:
# convert to wide format with separate column for each qualifier
pass_end_qualifiers = pass_end_qualifiers.pivot(index='event_id', columns='qualifier', values='value').reset_index()
display(pass_end_qualifiers)

qualifier,event_id,Pass End X,Pass End Y
0,2423549063,28.5,65.0
1,2423549097,37.2,93.6
2,2423549113,70.4,88.2
3,2423549127,60.6,87.9
4,2423549153,54.7,80.9
...,...,...,...
1057,2423665915,98.3,54.2
1058,2423667729,59.2,48.4
1059,2423668971,53.6,4.9
1060,2423669109,90.1,33.0


In [41]:
pass_end_qualifiers['Pass End X'] = pass_end_qualifiers['Pass End X'].astype(float)
pass_end_qualifiers['Pass End Y'] = pass_end_qualifiers['Pass End Y'].astype(float)

In [42]:
pass_end_qualifiers.head()

qualifier,event_id,Pass End X,Pass End Y
0,2423549063,28.5,65.0
1,2423549097,37.2,93.6
2,2423549113,70.4,88.2
3,2423549127,60.6,87.9
4,2423549153,54.7,80.9


In [45]:
# Add the qualifiers to the events
events = events_all.merge(pass_end_qualifiers,
                                     how='left',
                                     left_on='id',
                                     right_on='event_id')

display(events.head(2))

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,timeStamp,lastModified,qualifier,playerId,lineBreakingPass.linesBroken.value,passOption.player,passTarget.player,xThreat.applied,lineBreakingPass.lastLineBroken.value,pressure.pressureReceived.value,pressure.player,xThreat.removed,keyPass,assist,event_id,Pass End X,Pass End Y
0,2423529553,1,34,16,0,0,bx0cdmzr2gwr70ez72dorx82p,1,0.0,0.0,2022-05-21T18:06:19.198Z,2022-05-21T18:54:48Z,"[{'id': 3585971337, 'qualifierId': 59, 'value'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2423530567,1,34,16,0,0,3c3jcs7vc1t6vz5lev162jyv7,1,0.0,0.0,2022-05-21T18:09:00.087Z,2022-05-21T19:02:33Z,"[{'id': 3585976649, 'qualifierId': 194, 'value...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pass Data

In [46]:
# filter to only pass events
is_pass_event = events.typeId == 1
is_without_predictions = events['passOption.player'].isna()
remove_passes_without_predictions = events['passOption.player'].notna()
remove_passes_without_target_predictions = events['passTarget.player'].notna()

In [47]:
pass_events = events.loc[is_pass_event & remove_passes_without_predictions & remove_passes_without_target_predictions]
pass_events_without_predictions = events.loc[is_pass_event & is_without_predictions]

In [48]:
print("There are " + str(len(pass_events)) + " passes **with** predictions")
print("There are " + str(len(pass_events_without_predictions)) + " set piece passes without predictions")

There are 931 passes **with** predictions
There are 93 set piece passes without predictions


In [49]:
# flatten json for pass options
pass_options = pass_events[['id', 'passOption.player']].explode('passOption.player')
display(pass_options.head())
pass_options_flat = pd.json_normalize(pass_options['passOption.player'])

# save corresponding event ids for each qualifier
event_ids = pass_options[pass_options['passOption.player'].notna()].id.tolist()
pass_options_flat['event_id'] = event_ids

,id,passOption.player
5,2423549097,"{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'shi..."
5,2423549097,"{'playerId': 'e7e68wlpiqqohpg71oh4vrbl6', 'shi..."
5,2423549097,"{'playerId': 'vja0xo3xiuax8eh0b6q3y09', 'shirt..."
5,2423549097,"{'playerId': '7cp51c8zn7y08iyk0hc9ix5nt', 'shi..."
5,2423549097,"{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'shi..."


In [ ]:
# merge on the player names
# player_cols = ['playerId']
# pass_options_flat = pass_options_flat.merge(player_info[player_cols].drop_duplicates(),
#                                             how='left',
#                                             on=['playerId'])

In [50]:
# flatten json for pass target
pass_targets = pass_events[['id', 'passTarget.player']].explode('passTarget.player')
pass_targets_flat = pd.json_normalize(pass_targets['passTarget.player'])

# save corresponding event ids for each qualifier
target_event_ids = pass_targets[pass_targets['passTarget.player'].notna()].id.tolist()
pass_targets_flat['event_id'] = target_event_ids

In [ ]:
# merge on the player names
# pass_targets_flat = pass_targets_flat.merge(player_info[player_cols].drop_duplicates(),
#                                             how='left',
#                                             on=['playerId'])

In [51]:
# join on the options and targets to the original passes
pass_events_with_info = (pass_events
                         .merge(pass_options_flat,
                                how='left',
                                on='event_id',
                                suffixes=("", "_option"))
                         .merge(pass_targets_flat,
                                how='left',
                                on='event_id',
                                suffixes=("", "_target"))
                        )

In [52]:
pass_events_with_info.head()

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,timeStamp,lastModified,qualifier,playerId,lineBreakingPass.linesBroken.value,passOption.player,passTarget.player,xThreat.applied,lineBreakingPass.lastLineBroken.value,pressure.pressureReceived.value,pressure.player,xThreat.removed,keyPass,assist,event_id,Pass End X,Pass End Y,playerId_option,shirtNumber,predictions.expectedPassReceiver.value,predictions.expectedPassCompletion.value,predictions.expectedThreat.value,predictions.passOptionQuality.value,playerId_target,shirtNumber_target,predictions.expectedPassReceiver.value_target,predictions.expectedPassCompletion.value_target,predictions.expectedThreat.value_target,predictions.passOptionQuality.value_target
0,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-21T18:59:37.443Z,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.423549e+09,37.2,93.6,e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.6592077613,0.3655609488,0.0272596478,-0.255,7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
1,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-21T18:59:37.443Z,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.423549e+09,37.2,93.6,e7e68wlpiqqohpg71oh4vrbl6,11,0.0037940145,0.2479383051,0.0191572905,-0.453,7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
2,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-21T18:59:37.443Z,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.423549e+09,37.2,93.6,vja0xo3xiuax8eh0b6q3y09,4,0.002256453,0.4868352115,0.0069071949,-0.472,7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
3,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-21T18:59:37.443Z,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.423549e+09,37.2,93.6,7cp51c8zn7y08iyk0hc9ix5nt,5,0.0033909082,0.9509418011,0.0005734563,-0.254,7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
4,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-21T18:59:37.443Z,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.423549e+09,37.2,93.6,5qgc6zjc38a5xjl35gs7h3vu1,7,0.6129552126,0.4009089768,0.026168257,-0.248,7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189


In [53]:
useful_pass_cols = ['id', 'typeId',
                    'periodId', 'timeMin', 'timeSec', 'playerId',
                    'outcome', 'x', 'y', 'Pass End X', 'Pass End Y', 'timeStamp', 'passOption.player',
                    'xThreat.applied', 'keyPass',
                    'assist', 'playerId_option', 'shirtNumber',
                    'predictions.expectedPassReceiver.value', 'predictions.expectedPassCompletion.value',
                    'predictions.expectedThreat.value', 'predictions.passOptionQuality.value', 'passTarget.player',
                    'playerId_target', 'shirtNumber_target', 'predictions.expectedPassReceiver.value_target',
                    'predictions.expectedPassCompletion.value_target', 'predictions.expectedThreat.value_target',
                    'predictions.passOptionQuality.value_target']

In [55]:
# tidy column names
passes = pass_events_with_info[useful_pass_cols].rename(columns={"typeId":"event_type_id",
                                                                 "id":"event_id",
                                                                 "periodId": "period",
                                                                 "timeMin": "minute",
                                                                 "timeSec": "second",
                                                                 "Pass End X": "endx",
                                                                 "Pass End Y": "endy",
                                                                 "xThreat.applied": "xThreat",
                                                                 "keyPass": "chance_created",
                                                                 "positionX": "x_option",
                                                                 "positionY": "y_option",
                                                                 "predictions.expectedPassReceiver.value": "xR_option",
                                                                 "predictions.expectedPassCompletion.value": "xP_option",
                                                                 "predictions.expectedThreat.value": "xT_option",
                                                                 "predictions.passOptionQuality.value": "pass_option_quality",
                                                                 "positionX_target": "x_target",
                                                                 "positionY_target": "y_target",
                                                                 "predictions.expectedPassReceiver.value_target": "xR_target",
                                                                 "predictions.expectedPassCompletion.value_target": "xP_target",
                                                                 "predictions.expectedThreat.value_target": "xT_target",
                                                                 "predictions.passOptionQuality.value_target": "pass_option_quality_target"
                                                                })

passes.head()

,event_id,event_type_id,period,minute,second,playerId,outcome,x,y,endx,endy,timeStamp,passOption.player,xThreat,chance_created,assist,playerId_option,shirtNumber,xR_option,xP_option,xT_option,pass_option_quality,passTarget.player,playerId_target,shirtNumber_target,xR_target,xP_target,xT_target,pass_option_quality_target
0,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.6592077613,0.3655609488,0.0272596478,-0.255,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
1,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,e7e68wlpiqqohpg71oh4vrbl6,11,0.0037940145,0.2479383051,0.0191572905,-0.453,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
2,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,vja0xo3xiuax8eh0b6q3y09,4,0.002256453,0.4868352115,0.0069071949,-0.472,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
3,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,7cp51c8zn7y08iyk0hc9ix5nt,5,0.0033909082,0.9509418011,0.0005734563,-0.254,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189
4,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,5qgc6zjc38a5xjl35gs7h3vu1,7,0.6129552126,0.4009089768,0.026168257,-0.248,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.9812835455,0.995218277,0.0029771626,-0.189


In [56]:
# assign columns as numeric
numeric_cols = ['xR_option', 'xP_option', 'xT_option', 'xR_target', 'xP_target', 'xT_target', 'x', 'y']
passes[numeric_cols] = passes[numeric_cols].apply(pd.to_numeric)

In [57]:
# assign pass columns
xReceiver_limit = 0.7

passes.loc[:, "is_realistic_pass"] = np.where(passes.xR_option >= xReceiver_limit,
                                              1, 0)

passes.loc[:, "is_target_pass"] = np.where(passes.playerId_target == passes.playerId_option,
                                           1, 0)

passes.loc[:, "is_realistic_and_target_pass"] = np.where((passes.playerId_target == passes.playerId_option) &
                                                         (passes.xR_option >= xReceiver_limit),
                                                         1, 0)

passes.loc[:, "starts_in_own_third"] = np.where(passes.x <= 33.33, 1, 0)

In [58]:
passes.head()

,event_id,event_type_id,period,minute,second,playerId,outcome,x,y,endx,endy,timeStamp,passOption.player,xThreat,chance_created,assist,playerId_option,shirtNumber,xR_option,xP_option,xT_option,pass_option_quality,passTarget.player,playerId_target,shirtNumber_target,xR_target,xP_target,xT_target,pass_option_quality_target,is_realistic_pass,is_target_pass,is_realistic_and_target_pass,starts_in_own_third
0,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.659208,0.365561,0.027260,-0.255,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,0,0,0,1
1,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,e7e68wlpiqqohpg71oh4vrbl6,11,0.003794,0.247938,0.019157,-0.453,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,0,0,0,1
2,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,vja0xo3xiuax8eh0b6q3y09,4,0.002256,0.486835,0.006907,-0.472,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,0,0,0,1
3,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,7cp51c8zn7y08iyk0hc9ix5nt,5,0.003391,0.950942,0.000573,-0.254,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,0,0,0,1
4,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,5qgc6zjc38a5xjl35gs7h3vu1,7,0.612955,0.400909,0.026168,-0.248,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,0,0,0,1


## Availability Analysis

In [59]:
availability_player_cols = ['playerId_option']

In [60]:
availability_analysis = (passes
                         .groupby(availability_player_cols)
                         .agg(total_passes_while_on_pitch = ('event_id', 'count'),
                              was_targeted = ('is_target_pass', 'sum'),
                              was_realistic_option = ('is_realistic_pass', 'sum')
                             )
                         .reset_index()
                        )

In [61]:
availability_analysis.loc[:, "targeted_perc"] = round(100*availability_analysis.was_targeted/
                                                      availability_analysis.total_passes_while_on_pitch,1)

availability_analysis.loc[:, "used_when_available_perc"] = round(100*availability_analysis.was_targeted/
                                                      availability_analysis.was_realistic_option,1)

availability_analysis.loc[:, "availability_perc"] = round(100*availability_analysis.was_realistic_option/
                                                          availability_analysis.total_passes_while_on_pitch,1)

In [62]:
availability_analysis.head()

,playerId_option,total_passes_while_on_pitch,was_targeted,was_realistic_option,targeted_perc,used_when_available_perc,availability_perc
0,2lvit204llltk13iglsa2tjah,53,3,7,5.7,42.9,13.2
1,3sc349yey596xp2j6xlyt0frp,368,47,99,12.8,47.5,26.9
2,3vx94h32ahujciraspdayj9t6,296,35,84,11.8,41.7,28.4
3,4u281v53ges3kimtgac0tidm2,471,39,64,8.3,60.9,13.6
4,5ak9fwtqlr2pll0nsv5br7p7u,139,14,36,10.1,38.9,25.9


In [63]:
# set filters for availablilty
# It would be better to use a minutes filter here
# 1364 is the average Ligue 1 passes per team per game for three games
is_mininum_passes_while_on_pitch = availability_analysis.total_passes_while_on_pitch >= 10

In [64]:
# let's look at the most available players
(availability_analysis
 .loc[is_mininum_passes_while_on_pitch]
 .sort_values(by='used_when_available_perc', ascending=False)
 .head(10)
)

,playerId_option,total_passes_while_on_pitch,was_targeted,was_realistic_option,targeted_perc,used_when_available_perc,availability_perc
13,7ty1wdxxnusgkl34k5raipbl5,42,7,9,16.7,77.8,21.4
6,6ekdnbnk56xlxforb5owt3dn9,381,22,30,5.8,73.3,7.9
22,azuc3tma44xyrbgf5y279o1xx,237,34,52,14.3,65.4,21.9
3,4u281v53ges3kimtgac0tidm2,471,39,64,8.3,60.9,13.6
10,7cp51c8zn7y08iyk0hc9ix5nt,452,44,73,9.7,60.3,16.2
7,6j0ogojh2b7poyceg7i3k09yi,455,57,99,12.5,57.6,21.8
8,6u2ob6fv950r1qve8uejkq2uh,461,74,130,16.1,56.9,28.2
20,afymbx9eo87zau8mo99pakbu,341,29,52,8.5,55.8,15.2
30,vja0xo3xiuax8eh0b6q3y09,319,33,60,10.3,55.0,18.8
17,96wcx761pzv5ub4sfwsynp51x,361,44,81,12.2,54.3,22.4


## Pass Predictions Analysis

In [65]:
# each pass has the options of passes to all other teammates. Filter to only 'realistic' passes.
is_realistic_pass = passes.is_realistic_pass == 1

In [66]:
filtered_passes = passes.loc[is_realistic_pass]

In [67]:
xP_average = filtered_passes.xP_target.mean()
xP_upper_quartile = filtered_passes.xP_target.quantile(0.75)
xT_average = filtered_passes.xT_target.mean()
xT_upper_quartile = filtered_passes.xT_target.quantile(0.75)
xT_upper_2nd_decile = filtered_passes.xT_target.quantile(0.8)
xT_upper_decile = filtered_passes.xT_target.quantile(0.9)

In [68]:
print("xP upper quartile = " + str(round(xP_upper_quartile,4)) +
      " & xT upper quartile = " + str(round(xT_upper_quartile,4)))

xP upper quartile = 0.9942 & xT upper quartile = 0.0506


In [69]:
# set new columns
filtered_passes.loc[:, "is_good_opportunity"] = np.where((filtered_passes.xT_option >= xT_upper_decile) &
                                                         (filtered_passes.xP_option >= 0.8) &
                                                         (filtered_passes.xR_option >= xReceiver_limit), 1, 0)

filtered_passes.loc[:, "good_opportunity_taken"] = np.where((filtered_passes.is_target_pass == 1) &
                                                            (filtered_passes.is_good_opportunity == 1), 1, 0)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15016\3798532512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_passes.loc[:, "is_good_opportunity"] = np.where((filtered_passes.xT_option >= xT_upper_decile) &
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15016\3798532512.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_passes.loc[:, "good_opportunity_taken"] = np.where((filtered_passes.is_target_pass == 1) &


In [70]:
filtered_passes.head()

,event_id,event_type_id,period,minute,second,playerId,outcome,x,y,endx,endy,timeStamp,passOption.player,xThreat,chance_created,assist,playerId_option,shirtNumber,xR_option,xP_option,xT_option,pass_option_quality,passTarget.player,playerId_target,shirtNumber_target,xR_target,xP_target,xT_target,pass_option_quality_target,is_realistic_pass,is_target_pass,is_realistic_and_target_pass,starts_in_own_third,is_good_opportunity,good_opportunity_taken
6,2423549097,1,1,0,2,azuc3tma44xyrbgf5y279o1xx,1,31.5,57.2,37.2,93.6,2022-05-21T18:59:37.443Z,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0029771626,NaN,NaN,7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,"[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",7sep6mx2s67mh5fr3raxu7aei,13,0.981284,0.995218,0.002977,-0.189,1,1,1,1,0,0
10,2423549113,1,1,0,7,7sep6mx2s67mh5fr3raxu7aei,1,49.2,95.4,70.4,88.2,2022-05-21T18:59:42.482Z,"[{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'sh...",0.0309752524,NaN,NaN,5qgc6zjc38a5xjl35gs7h3vu1,7,0.830498,0.764126,0.012516,-0.204,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.874496,0.574415,0.030975,-0.061,1,0,0,0,0,0
12,2423549113,1,1,0,7,7sep6mx2s67mh5fr3raxu7aei,1,49.2,95.4,70.4,88.2,2022-05-21T18:59:42.482Z,"[{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'sh...",0.0309752524,NaN,NaN,6u2ob6fv950r1qve8uejkq2uh,1,0.897717,0.579454,0.024249,-0.156,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.874496,0.574415,0.030975,-0.061,1,0,0,0,0,0
15,2423549113,1,1,0,7,7sep6mx2s67mh5fr3raxu7aei,1,49.2,95.4,70.4,88.2,2022-05-21T18:59:42.482Z,"[{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'sh...",0.0309752524,NaN,NaN,e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.874496,0.574415,0.030975,-0.061,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",e3kdoxu1kwn2w3wwi1rqhvr9x,17,0.874496,0.574415,0.030975,-0.061,1,1,1,0,0,0
25,2423549127,1,1,0,9,e3kdoxu1kwn2w3wwi1rqhvr9x,1,72.1,88.0,60.6,87.9,2022-05-21T18:59:43.755Z,"[{'playerId': '7cp51c8zn7y08iyk0hc9ix5nt', 'sh...",0.0338825583,NaN,NaN,5qgc6zjc38a5xjl35gs7h3vu1,7,0.864377,0.802335,0.013744,-0.16,"[{'playerId': '6u2ob6fv950r1qve8uejkq2uh', 'sh...",6u2ob6fv950r1qve8uejkq2uh,1,0.962613,0.697703,0.033883,0.065,1,0,0,0,0,0


In [71]:
group_pass_cols = ['event_id', 'playerId', 'outcome', 'period', 'minute',
                   'second', 'x', 'y', 'endx', 'endy', 'playerId_target',
                   'xR_target', 'xP_target', 'xT_target']

pass_analysis = (filtered_passes
                 .groupby(group_pass_cols)
                 .agg(options = ('playerId_option', 'count'),
                      good_passing_opportunites = ('is_good_opportunity', 'sum'),
                      good_passing_opportunites_taken = ('good_opportunity_taken', 'sum'),
                      xR_max = ('xR_option', 'max'),
                      xR_min = ('xR_option', 'min'),
                      xP_max = ('xP_option', 'max'),
                      xP_min = ('xP_option', 'min'),
                      xT_max = ('xT_option', 'max'),
                      xT_min = ('xT_option', 'min')
                     )
                 .reset_index()
                )

In [72]:
pass_analysis.loc[:, "is_safest_pass"] = np.where(pass_analysis.xP_target >= pass_analysis.xP_max, 1, 0)
pass_analysis.loc[:, "is_most_dangerous_pass"] = np.where(pass_analysis.xT_target >= pass_analysis.xT_max, 1, 0)
pass_analysis.loc[:, "is_safe_pass"] = np.where(pass_analysis.xP_target >= xP_upper_quartile, 1, 0)
pass_analysis.loc[:, "is_dangerous_pass"] = np.where(pass_analysis.xT_target >= xT_upper_quartile, 1, 0)
pass_analysis.loc[:, "is_unpredictable_pass"] = np.where(pass_analysis.xR_target < xReceiver_limit, 1, 0)
pass_analysis.loc[:, "good_passing_opportunites_missed"] = np.where((pass_analysis.good_passing_opportunites > 0) &
                                                                (pass_analysis.good_passing_opportunites_taken == 0) &
                                                                    (pass_analysis.is_most_dangerous_pass == 0) &
                                                                    #(pass_analysis.is_dangerous_pass == 0)
                                                                    (pass_analysis.xT_target >= xT_upper_decile)
                                                                    ,
                                                                    1, 0)

## Overall Player Pass Analysis

Overall summaries of passes taken by players

In [73]:
passes.loc[:, "is_good_opportunity"] = np.where((passes.xT_option >= xT_upper_decile) &
                                                (passes.xP_option >= 0.8) &
                                                (passes.xR_option >= xReceiver_limit), 1, 0)

passes.loc[:, "good_opportunity_taken"] = np.where((passes.is_target_pass == 1) &
                                                   (passes.is_good_opportunity == 1), 1, 0)

all_pass_analysis = (passes
                     .groupby(group_pass_cols)
                     .agg(
                          options = ('playerId_option', 'count'),
                          good_passing_opportunites = ('is_good_opportunity', 'sum'),
                          good_passing_opportunites_taken = ('good_opportunity_taken', 'sum'),
                          xR_max = ('xR_option', 'max'),
                          xR_min = ('xR_option', 'min'),
                          xP_max = ('xP_option', 'max'),
                          xP_min = ('xP_option', 'min'),
                          xT_max = ('xT_option', 'max'),
                          xT_min = ('xT_option', 'min')
                     )
                 .reset_index()
                )

In [74]:
all_pass_analysis.loc[:, "is_safest_pass"] = np.where(all_pass_analysis.xP_target >= all_pass_analysis.xP_max, 1, 0)
all_pass_analysis.loc[:, "is_most_dangerous_pass"] = np.where(all_pass_analysis.xT_target >= all_pass_analysis.xT_max, 1, 0)
all_pass_analysis.loc[:, "is_safe_pass"] = np.where(all_pass_analysis.xP_target >= xP_upper_quartile, 1, 0)
all_pass_analysis.loc[:, "is_dangerous_pass"] = np.where(all_pass_analysis.xT_target >= xT_upper_quartile, 1, 0)
all_pass_analysis.loc[:, "is_unpredictable_pass"] = np.where(all_pass_analysis.xR_target < xReceiver_limit, 1, 0)
all_pass_analysis.loc[:, "good_passing_opportunites_missed"] = np.where((all_pass_analysis.good_passing_opportunites > 0) &
                                                                (all_pass_analysis.good_passing_opportunites_taken == 0) &
                                                                    (all_pass_analysis.is_most_dangerous_pass == 0) &
                                                                    #(all_pass_analysis.is_dangerous_pass == 0),
                                                                    (all_pass_analysis.xT_target >= xT_upper_decile),
                                                                    1, 0)

In [75]:
player_cols = [ 'playerId']

In [76]:
player_analysis_all_passes = (all_pass_analysis
                              .groupby(player_cols)
                              .agg(total_passes = ('event_id', 'count'),
                                  )
                              .reset_index()
                             )

## Typical Player Pass Analysis

In [77]:
player_cols = ['playerId']

In [78]:
player_analysis = (pass_analysis
                   .groupby(player_cols)
                   .agg(total_passes = ('event_id', 'count'),
                        safe_passes_taken = ('is_safe_pass', 'sum'),
                        dangerous_option_taken = ('is_dangerous_pass', 'sum'),
                        unpredictable_option_taken = ('is_unpredictable_pass', 'sum'),
                        safest_pass_taken = ('is_safest_pass', 'sum'),
                        most_dangerous_option_taken = ('is_most_dangerous_pass', 'sum'),
                        good_passing_opportunites_missed = ('good_passing_opportunites_missed', 'sum'),
                        good_passing_opportunites_taken = ('good_passing_opportunites_taken', 'sum'),
                        average_options = ('options', 'mean'),
                        average_xR = ('xR_target', 'mean'),
                        average_xP = ('xP_target', 'mean'),
                        average_xT = ('xT_target', 'mean')
                       )
                   .reset_index()
                  )

In [79]:
player_analysis.head()

,playerId,total_passes,safe_passes_taken,dangerous_option_taken,unpredictable_option_taken,safest_pass_taken,most_dangerous_option_taken,good_passing_opportunites_missed,good_passing_opportunites_taken,average_options,average_xR,average_xP,average_xT
0,2lvit204llltk13iglsa2tjah,1,0,0,0,0,1,0,0,2.000000,0.750543,0.363136,0.006751
1,3sc349yey596xp2j6xlyt0frp,46,11,4,4,32,14,0,1,2.195652,0.857108,0.861019,0.017474
2,3vx94h32ahujciraspdayj9t6,18,2,4,1,13,13,0,0,1.944444,0.901199,0.845720,0.032589
3,4u281v53ges3kimtgac0tidm2,44,18,5,7,29,26,0,0,1.863636,0.862444,0.861458,0.017690
4,5ak9fwtqlr2pll0nsv5br7p7u,17,0,12,3,10,9,0,1,2.058824,0.823692,0.585973,0.111240


In [80]:
player_analysis.loc[:, "safest_pass_perc"] = round(100*player_analysis.safest_pass_taken/
                                                   player_analysis.total_passes,1)

player_analysis.loc[:, "most_dangerous_option_perc"] = round(100*player_analysis.most_dangerous_option_taken/
                                                             player_analysis.total_passes,1)

player_analysis.loc[:, "unpredictable_option_perc"] = round(100*player_analysis.unpredictable_option_taken/
                                                             player_analysis.total_passes,1)

player_analysis.loc[:, "safe_pass_perc"] = round(100*player_analysis.safe_passes_taken/
                                                 player_analysis.total_passes,1)

player_analysis.loc[:, "dangerous_option_perc"] = round(100*player_analysis.dangerous_option_taken/
                                                        player_analysis.total_passes,1)

player_analysis.loc[:,
                    "good_passing_opportunites_missed_perc"] = round(100*
                                                                     player_analysis.good_passing_opportunites_missed/
                                                                     (player_analysis.good_passing_opportunites_missed +
                                                                      player_analysis.good_passing_opportunites_taken),1)

## Summary Stats

In [81]:
def opta_vision_summaries(data):
    
    # create summarise of given dataset
    total_passes = data.event_id.nunique()
    safest_passes = data.is_safest_pass.sum()
    most_dangerous_passes = data.is_most_dangerous_pass.sum()
    safe_passes = data.is_safe_pass.sum()
    dangerous_passes = data.is_dangerous_pass.sum()
    unpredictable_passes = data.is_unpredictable_pass.sum()
    average_options = data.options.mean().round(1)
    safest_pass_perc = round(100*safest_passes/total_passes,1)
    most_dangerous_pass_perc = round(100*most_dangerous_passes/total_passes,1)
    safe_pass_perc = round(100*safe_passes/total_passes,1)
    dangerous_pass_perc = round(100*dangerous_passes/total_passes,1)
    unpredictable_pass_perc = round(100*unpredictable_passes/total_passes,1)
    missed_opportunites = data.good_passing_opportunites_missed.sum()
    good_opportunites = (data.good_passing_opportunites_taken.sum()+data.good_passing_opportunites_missed.sum())
    missed_opp_perc = round(100*missed_opportunites/good_opportunites,1)
    

    print(f"There are {total_passes} total passes")
    print(f"On average, a player has {average_options} options when they make a pass")
    print(f"A player takes the safest option {safest_pass_perc}% of the time")
    print(f"A player takes the most dangerous option {most_dangerous_pass_perc}% of the time")
    print(f"A player takes a safe option {safe_pass_perc}% of the time")
    print(f"A player takes a dangerous option {dangerous_pass_perc}% of the time")
    print(f"A player takes an unlikely option {unpredictable_pass_perc}% of the time")
    print(f"Good opportunies are missed {missed_opp_perc}% of the time")

In [82]:
print("ALL PASSES:")
opta_vision_summaries(all_pass_analysis)

ALL PASSES:
There are 931 total passes
On average, a player has 10.0 options when they make a pass
A player takes the safest option 17.9% of the time
A player takes the most dangerous option 11.3% of the time
A player takes a safe option 28.7% of the time
A player takes a dangerous option 24.6% of the time
A player takes an unlikely option 16.0% of the time
Good opportunies are missed 24.3% of the time


In [83]:
print("REALISTIC PASSES:")
opta_vision_summaries(pass_analysis)

REALISTIC PASSES:
There are 912 total passes
On average, a player has 2.0 options when they make a pass
A player takes the safest option 56.8% of the time
A player takes the most dangerous option 53.1% of the time
A player takes a safe option 28.7% of the time
A player takes a dangerous option 24.7% of the time
A player takes an unlikely option 14.3% of the time
Good opportunies are missed 17.6% of the time


## Final Player Outputs

In [84]:
player_id_list = events_all.playerId.dropna().unique().tolist()[:3]

In [85]:
is_chosen_players = player_analysis.playerId.isin(player_id_list)
is_merged_players = player_analysis_all_passes.playerId.isin(player_id_list)
is_availability_players = availability_analysis.playerId_option.isin(player_id_list)

In [86]:
merged_cols = ['playerId']

In [87]:
pressure_cols = ['playerId', 'playerName',
                 'pressures', 'high_pressures', 'high_pressure_perc'
                ]

In [88]:
option_cols = ['playerId', 'total_passes',
               'average_options',
               'safest_pass_perc', 'most_dangerous_option_perc', 'unpredictable_option_perc', 'safe_pass_perc',
               'dangerous_option_perc', 'good_passing_opportunites_missed_perc']

In [89]:
availability_cols = ['playerId_option',
                     'total_passes_while_on_pitch', 
                     'targeted_perc', 
                     'used_when_available_perc', 
                     'availability_perc']

In [90]:
player_analysis.loc[is_chosen_players][option_cols]

,playerId,total_passes,average_options,safest_pass_perc,most_dangerous_option_perc,unpredictable_option_perc,safe_pass_perc,dangerous_option_perc,good_passing_opportunites_missed_perc
8,6u2ob6fv950r1qve8uejkq2uh,56,1.928571,67.9,60.7,14.3,19.6,58.9,12.5
12,7sep6mx2s67mh5fr3raxu7aei,37,2.054054,62.2,48.6,10.8,21.6,45.9,20.0
21,azuc3tma44xyrbgf5y279o1xx,39,1.615385,59.0,71.8,15.4,66.7,2.6,NaN


In [91]:
availability_analysis.loc[is_availability_players][availability_cols]

,playerId_option,total_passes_while_on_pitch,targeted_perc,used_when_available_perc,availability_perc
8,6u2ob6fv950r1qve8uejkq2uh,461,16.1,56.9,28.2
12,7sep6mx2s67mh5fr3raxu7aei,238,13.4,50.0,26.9
22,azuc3tma44xyrbgf5y279o1xx,237,14.3,65.4,21.9


## Insights

In [92]:
is_mininum_passes = player_analysis.total_passes >= 10

In [93]:
(player_analysis
 .loc[is_mininum_passes][option_cols]
 .sort_values(by='safest_pass_perc',
              ascending=True)
 .head(10)
)

,playerId,total_passes,average_options,safest_pass_perc,most_dangerous_option_perc,unpredictable_option_perc,safe_pass_perc,dangerous_option_perc,good_passing_opportunites_missed_perc
25,e3kdoxu1kwn2w3wwi1rqhvr9x,12,1.750000,33.3,83.3,16.7,0.0,91.7,NaN
19,afymbx9eo87zau8mo99pakbu,30,2.000000,33.3,56.7,20.0,0.0,40.0,0.0
24,dxb1r4gqgxkngb0pzvfby9iol,11,2.000000,36.4,54.5,18.2,27.3,45.5,NaN
16,96wcx761pzv5ub4sfwsynp51x,53,2.452830,39.6,43.4,18.9,9.4,17.0,33.3
15,8qmm84tue6kuz8e5nhhdhmz8p,35,2.257143,48.6,42.9,11.4,54.3,2.9,0.0
17,976riwm0dz0e74d4l28y3ttcl,51,2.058824,49.0,37.3,23.5,25.5,13.7,0.0
29,vja0xo3xiuax8eh0b6q3y09,41,1.975610,53.7,56.1,12.2,22.0,12.2,0.0
10,7cp51c8zn7y08iyk0hc9ix5nt,63,1.873016,54.0,55.6,15.9,57.1,7.9,NaN
26,e6ok0deqkoe80184iu509gzu2,29,2.241379,55.2,48.3,6.9,6.9,24.1,NaN
22,bvbebtykj45j3luvemk8yc4ph,36,2.027778,55.6,44.4,11.1,27.8,16.7,100.0


In [94]:
(player_analysis
 .loc[is_mininum_passes][option_cols]
 .sort_values(by='most_dangerous_option_perc',
              ascending=False)
 .head(10)
)

,playerId,total_passes,average_options,safest_pass_perc,most_dangerous_option_perc,unpredictable_option_perc,safe_pass_perc,dangerous_option_perc,good_passing_opportunites_missed_perc
25,e3kdoxu1kwn2w3wwi1rqhvr9x,12,1.750000,33.3,83.3,16.7,0.0,91.7,NaN
13,8f3bhiy6r5eei1n25exhbwr8p,20,2.000000,60.0,80.0,15.0,40.0,40.0,NaN
2,3vx94h32ahujciraspdayj9t6,18,1.944444,72.2,72.2,5.6,11.1,22.2,NaN
21,azuc3tma44xyrbgf5y279o1xx,39,1.615385,59.0,71.8,15.4,66.7,2.6,NaN
27,e7e68wlpiqqohpg71oh4vrbl6,21,1.714286,71.4,66.7,23.8,9.5,42.9,0.0
8,6u2ob6fv950r1qve8uejkq2uh,56,1.928571,67.9,60.7,14.3,19.6,58.9,12.5
3,4u281v53ges3kimtgac0tidm2,44,1.863636,65.9,59.1,15.9,40.9,11.4,NaN
18,a56woizbe4g6jpl3fg4tlgno5,17,2.000000,70.6,58.8,0.0,70.6,0.0,NaN
19,afymbx9eo87zau8mo99pakbu,30,2.000000,33.3,56.7,20.0,0.0,40.0,0.0
29,vja0xo3xiuax8eh0b6q3y09,41,1.975610,53.7,56.1,12.2,22.0,12.2,0.0
